In [ ]:
import os
from netrc import netrc
from subprocess import Popen
from platform import system
from getpass import getpass

from pystac_client import Client  
from pystac_client import ItemSearch
from pystac.item import Item
from typing import Dict, Any
import json

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from datetime import datetime
from tqdm import tqdm

from shapely.geometry import box
from shapely.geometry import shape
from shapely.ops import transform

import numpy as np
import pandas as pd
import geopandas as gpd
from skimage import io

from osgeo import gdal
import rasterio as rio

import pyproj
from pyproj import Proj

import xarray as xr
import panel as pn
import geoviews as gv
import hvplot.xarray
import holoviews as hv

from bokeh.models import FixedTicker
hv.extension('bokeh')
gv.extension('bokeh', 'matplotlib')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
inDir = os.getcwd()
os.chdir(inDir)

In [ ]:
# Generates authentication token
# Asks for your Earthdata username and password for first time, if netrc does not exists.

urs = 'urs.earthdata.nasa.gov'    # Earthdata URL endpoint for authentication
prompts = ['Enter NASA Earthdata Login Username: ',
           'Enter NASA Earthdata Login Password: ']

# Determine the OS (Windows machines usually use an '_netrc' file)
netrc_name = "_netrc" if system()=="Windows" else ".netrc"

# Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
try:
    netrcDir = os.path.expanduser(f"~/{netrc_name}")
    netrc(netrcDir).authenticators(urs)[0]

# Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
except FileNotFoundError:
    homeDir = os.path.expanduser("~")
    Popen('touch {0}{2} | echo machine {1} >> {0}{2}'.format(homeDir + os.sep, urs, netrc_name), shell=True)
    Popen('echo login {} >> {}{}'.format(getpass(prompt=prompts[0]), homeDir + os.sep, netrc_name), shell=True)
    Popen('echo \'password {} \'>> {}{}'.format(getpass(prompt=prompts[1]), homeDir + os.sep, netrc_name), shell=True)
    # Set restrictive permissions
    Popen('chmod 0600 {0}{1}'.format(homeDir + os.sep, netrc_name), shell=True)

    # Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
except TypeError:
    homeDir = os.path.expanduser("~")
    Popen('echo machine {1} >> {0}{2}'.format(homeDir + os.sep, urs, netrc_name), shell=True)
    Popen('echo login {} >> {}{}'.format(getpass(prompt=prompts[0]), homeDir + os.sep, netrc_name), shell=True)
    Popen('echo \'password {} \'>> {}{}'.format(getpass(prompt=prompts[1]), homeDir + os.sep, netrc_name), shell=True)

In [ ]:
# GDAL configurations used to successfully access PODAAC Cloud Assets via vsicurl 
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')

In [ ]:
# USER-DEFINED PARAMETERS
aoi = box(67.4, 26.2, 68.0, 26.8)
start_date = datetime(2022, 1, 1)                                       # in 2022-01-01 00:00:00 format
stop_date = f"{datetime.today().strftime('%Y-%m-%d')} 23:59:59"         # in 2022-01-01 00:00:00 format
overlap_threshold = 50                                                  # in percent
#cloud_cover_threshold = 40                                              # in percent

print(f"Search between {start_date} and {stop_date}")
print(f"With AOI: {aoi.__geo_interface__}")

In [ ]:
# Search data through CMR-STAC API
stac = 'https://cmr.earthdata.nasa.gov/cloudstac/'    # CMR-STAC API Endpoint
api = Client.open(f'{stac}/POCLOUD/')
collections = ['OPERA_L3_DSWX-HLS_PROVISIONAL_V0']

search_params = {"collections": collections,
                 "intersects": aoi.__geo_interface__,
                 "datetime": [start_date, stop_date],
                 "max_items": 1000}
search_dswx = api.search(**search_params)

In [ ]:
# Function to calculate percentage overlap between user-defined bbox and dswx tile
def intersection_percent(item: Item, aoi: Dict[str, Any]) -> float:
    '''The percentage that the Item's geometry intersects the AOI. An Item that
    completely covers the AOI has a value of 100.
    '''
    geom_item = shape(item.geometry)
    geom_aoi = shape(aoi)
    intersected_geom = geom_aoi.intersection(geom_item)
    intersection_percent = (intersected_geom.area * 100) / geom_aoi.area

    return intersection_percent

In [ ]:
# Filter datasets based on spatial overlap and cloud cover
intersects_geometry = aoi.__geo_interface__

#Check percent overlap values
print("Percent overlap before filtering: ")
print([f"{intersection_percent(i, intersects_geometry):.2f}" for i in search_dswx.items()])

# Apply spatial overlap
dswx_filtered = (
    i for i in search_dswx.items() if intersection_percent(i, intersects_geometry) > overlap_threshold 
)

In [ ]:
# Inspect the items inside the filltered query
dswx_data = list(dswx_filtered)
# Inspect one data
dswx_data[0].to_dict()

In [ ]:
## Print search information
# Tota granules
print(f"Total granules after search filter: {len(dswx_data)}")

#Check percent overlap values
print("Percent-overlap: ")
print([f"{intersection_percent(i, intersects_geometry):.2f}" for i in dswx_data])

In [ ]:
geom_df = []
for d,_ in enumerate(dswx_data):
    geom_df.append(shape(dswx_data[d].geometry))

geom_granules = gpd.GeoDataFrame({'geometry':geom_df})
granules_poly = gv.Polygons(geom_granules, label='DSWx tile boundary').opts(line_color='blue', color=None, show_legend=True)

# Use geoviews to combine a basemap with the shapely polygon of our Region of Interest (ROI)
base = gv.tile_sources.EsriImagery.opts(width=1000, height=1000)

# Get the user-specified aoi
geom_aoi = shape(intersects_geometry)
aoi_poly = gv.Polygons(geom_aoi, label='User-specified bbox').opts(line_color='yellow', color=None, show_legend=True)

# Plot using geoviews wrapper
granules_poly*base*aoi_poly

In [ ]:
print([f"{i.properties['datetime']}" for i in dswx_data])

In [ ]:
dswx_data_df = []
for item in dswx_data:
    item.to_dict()
    ID = item.id
    dat = item.datetime.strftime('%Y-%m-%d')
    spatial_coverage = intersection_percent(item, intersects_geometry)
    geom = item.geometry
    try:
        thumb = item.assets['browse'].href
    except KeyError:
        thumb = item.assets['0_BROWSE'].href

    # Take only B01-WTR, B02-BWTR, B03-CONF layers
    band_links = [item.assets['0_B01_WTR'].href, item.assets['0_B02_BWTR'].href, item.assets['0_B03_CONF'].href]
    dswx_data_df.append([dat,ID,geom,spatial_coverage,band_links,thumb])

dswx_data_df = pd.DataFrame(dswx_data_df, columns = ['Date', 'ID', 'Footprint','Spatial_Coverage','Band_Links','Thumbnail'])
dswx_data_df

In [ ]:
# Check out one of the dataset
dswx_data_df.iloc[1].Band_Links

In [ ]:
# Function to read each layers and stack them to create a geocube
def stack_bands(bandpath:str, bandlist:list): 
    '''
    Returns geocube with three bands stacked into one multi-dimensional array.
            Parameters:
                    bandpath (str): Path to bands that should be stacked
                    bandlist (list): Three bands that should be stacked
            Returns:
                    bandStack (xarray Dataset): Geocube with stacked bands
                    crs (int): Coordinate Reference System corresponding to bands
    '''
    bandStack = []; bandS = []; bandStack_ = [];
    for i,band in enumerate(bandlist):
        if i==0:
            bandStack_ = xr.open_rasterio(bandpath%band)
            crs = pyproj.CRS.to_epsg(pyproj.CRS.from_proj4(bandStack_.crs))
            bandStack_ = bandStack_ * bandStack_.scales[0]
            bandStack = bandStack_.squeeze(drop=True)
            bandStack = bandStack.to_dataset(name='z')
            bandStack.coords['band'] = i+1
            bandStack = bandStack.rename({'x':'longitude', 'y':'latitude', 'band':'band'})
            bandStack = bandStack.expand_dims(dim='band')  
        else:
            bandS = xr.open_rasterio(bandpath%band)
            bandS = bandS * bandS.scales[0]
            bandS = bandS.squeeze(drop=True)
            bandS = bandS.to_dataset(name='z')
            bandS.coords['band'] = i+1
            bandS = bandS.rename({'x':'longitude', 'y':'latitude', 'band':'band'})
            bandS = bandS.expand_dims(dim='band')
            bandStack = xr.concat([bandStack, bandS], dim='band')
    return bandStack, crs

In [ ]:
data_dir = dswx_data_df.iloc[1].Band_Links[0].split('v0.0')[0]
bandlist = ['B01_WTR', 'B02_BWTR', 'B03_CONF']
bandpath = f"{data_dir}v0.0_%s.tif"
bandpath

In [ ]:
# Creates geocube of stacked bands
da, crs = stack_bands(bandpath, bandlist)

# Creates basemap using geoviews
base = gv.tile_sources.EsriImagery.opts(width=1000, height=1000, padding=0.1)

In [ ]:
# Mask nodata values (255)
da_masked = da.where(da['z'] != 255.) 
B01_WTR = da_masked.z.sel({'band':1})
B02_BWTR = da_masked.z.sel({'band':2}) 
B03_CONF = da_masked.z.sel({'band':3}) 

In [ ]:
# Vizualize B01 - WATER CLASSIFICATION LAYER
# Parameters for Colorbar
levels = [0, 0.9, 1.9, 2.9, 7.9, 8.9, 10]
color_key = {
    "Not Water": "#ffffff",
    "Open Water": "#0000ff",
    "Partial Surface Water": "#00ff00",
    "Reserved": "#000000",
    "Snow/Ice": "#00ffff",
    "Clouds/Cloud Shadow": "#7f7f7f"
}

ticks = [0.5, 1.5, 2.5, 5.5, 8.5, 9.5]
ticker = FixedTicker(ticks=ticks)
labels = dict(zip(ticks, color_key))

fig = B01_WTR.hvplot.image(x='longitude', 
                          y='latitude', 
                          crs=crs, 
                          rasterize=False, 
                          dynamic=False, 
                          aspect='equal', 
                          frame_width=500,
                          clim=(0,255),
                          frame_height=500, 
                          alpha=0.8).opts(title=f"B01 WTR", xlabel='Longitude', ylabel='Latitude', color_levels= levels, cmap=tuple(color_key.values()), 
                                        colorbar_opts={'ticker': ticker, 'major_label_overrides': labels}, clim=(0,10)) * base
#Uncomment to save
#hvplot.save(fig, 'B01_WTR.png')
fig

In [ ]:
# Vizualize B02 - BINARY WATER LAYER
# Parameters for Colorbar
levels = [0, 0.9, 1.9, 7.9, 8.9, 10]
color_key = {
    "Not Water": "#ffffff",
    "Open Water": "#0000ff",
    "Reserved": "#000000",
    "Snow/Ice": "#00ffff",
    "Clouds/Cloud Shadow": "#7f7f7f"
}

ticks = [0.5, 1.5, 5.5, 8.5, 9.5]
ticker = FixedTicker(ticks=ticks)
labels = dict(zip(ticks, color_key))

fig = B02_BWTR.hvplot.image(x='longitude', 
                          y='latitude', 
                          crs=crs, 
                          rasterize=False, 
                          dynamic=False, 
                          aspect='equal', 
                          frame_width=500,
                          clim=(0,255),
                          frame_height=500, 
                          alpha=0.8).opts(title=f"B02 BWTR", xlabel='Longitude', ylabel='Latitude', color_levels= levels, cmap=tuple(color_key.values()), 
                                        colorbar_opts={'ticker': ticker, 'major_label_overrides': labels}, clim=(0,10)) * base
#Uncomment to save
#hvplot.save(fig, 'B02_BWTR.png')
fig

In [ ]:
# Vizualize B03 - CONFIDENCE LAYER
# Parameters for Colorbar
with rio.open(f"{data_dir}v0.0_B03_CONF.tif") as ds:
    colormap = ds.colormap(1)

color_key = ListedColormap([np.array(colormap[key]) / 255 for key in range(256)])
ticks = [0, 50, 100, 175, 245, 255]
ticker = FixedTicker(ticks=ticks)
labels = dict(zip(ticks, ["0", "Confidence", "100", "Reserved", "Snow/Ice", "Clouds/Cloud Shadow"]))

fig = B03_CONF.hvplot.image(x='longitude', 
                          y='latitude', 
                          crs=crs, 
                          rasterize=False, 
                          dynamic=False, 
                          aspect='equal', 
                          frame_width=500,
                          clim=(0,255),
                          frame_height=500, 
                          alpha=0.8).opts(title=f"B03 CONF", xlabel='Longitude', ylabel='Latitude', cmap=color_key,
                                    colorbar_opts={'ticker': ticker, 'major_label_overrides': labels}) * base
#Uncomment to save
#hvplot.save(fig, 'B03_CONF.png')
fig